** folked from kaggle.com/code/odins0n/nbme-detailed-eda here,**

** please give him an upvote**

** version 4 ** GroupBy added


  # <center> NBME Competition basic understand </center>
## <center>If you find this notebook helpful, support an upvote👍</center>

In [ ]:
import os
import spacy
import warnings
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
train = pd.read_csv("../input/nbme-score-clinical-patient-notes/train.csv")
test = pd.read_csv("../input/nbme-score-clinical-patient-notes/test.csv")
features = pd.read_csv("../input/nbme-score-clinical-patient-notes/features.csv")
patient_notes = pd.read_csv("../input/nbme-score-clinical-patient-notes/patient_notes.csv")
submission = pd.read_csv("../input/nbme-score-clinical-patient-notes/sample_submission.csv")

warnings.filterwarnings('ignore')

  # <center> 1. What should we predict on Model </center>


<h3> train consists of 6 items which are 'id, case_num, pn_num, feature_num, annotation, location' </h3>

In [ ]:
train.head(2)

<h3> test consits of 4 items which are 'id, case_num, pn_num, feature_num' </h3>

In [ ]:
test.head(2)


# <center> So, (annotation, location) is target what we should predict well  </center>

  # <center> 2. Understand Submission.csv </center>


In [ ]:
submission.head(3)

In [ ]:
test.head(3)

# <center> We should fill up location value in submisson.csv  </center>

**Three things we can get infos from submission id:**
* `case_num` - case_num has range between 0 ~ 9, each num belongs their group.
* `pn_num` - through pn_num, we can get Scripts at Patient_note.csv which is 'pn_history'
* `feature_num` - through feature_num, we can get Symptom at Features.csv which is 'feature_text'

In [ ]:
test = test.merge(features, on=['feature_num', 'case_num'], how='left')
test = test.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test.head())

In [ ]:
train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(train.head())

<h3> To understand test data conveniently, we merged every relevants. </h3>
<h3> So, we need to predict annotation based on Scripts (pn_history) and Symptoms (feature_text) </h3>

<h3> Also We can use groupby func in pandas to understand data directly </h3>
<h4> Using Pandas API is key to understand datas and also helpful many ways :)</h4>

In [ ]:
train_case_num_group = train.groupby(train['case_num'])
train_feature_num_group = train.groupby(train['feature_num'])

In [ ]:
for case_num, each_case in train_case_num_group:    
    print('case_num:', case_num)
    display(each_case.head(3))

In [ ]:
for feature_num, each_case in train_feature_num_group:    
    print('feature_num:', feature_num)
    display(each_case.head(3))
    break

<h4> Displayng feature_num groups skipped because feature_num counts is so big.</h4>

  # <center> 3. Understand feature_text, pn_history </center>

* `feature_text` - specific feature_text is located in Features.csv which matched at feature_num
                   - I named it as Symptoms. It describes a unique each symptoms.
                   
                   
* `pn_history` - specific pn_history is located in Patient_note.csv which matched at pn_num
                 - I named it as Scripts. It describes patient state
                 - And we need to find matched features (annotation) in Scripts.                                
         

### Dataframe for a particular patient

In [ ]:
PATIENT_IDX = 74087
patient_df = train[train["pn_num"] == PATIENT_IDX]
patient_df

In [ ]:
PATIENT_IDX = 74087
patient_df = train[train["pn_num"] == PATIENT_IDX]
patient_df['pn_history'][10799]

In [ ]:
features[features['feature_num'] == 708]

<h3> For example, if test id is 74087_708 </h3>
<h3> we need to find annotation '6 months' in pn_history Scripts which located at Patient_notes.csv where pn_num = 74087 </h3>
<h3> also need to consider 708 Features in Feature.csv which express that 'symptoms-for-6-months'</h3>

# <center> 4. Desired Predicted Case </center>

<h4> we should, for example, predict index location 'initermittent episodes' located </h4>
<h4> and this is refered 'Intermittent symptoms' in Features.csv </h4>

* image copied from (https://www.kaggle.com/code/theoviel/roberta-strikes-back) thanks!

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img

image = img.imread('../input/picture/result.PNG')
plt.figure(figsize = (200,200))
plt.imshow(image)
plt.show()

# <center> any errors or comments will be appreciate it :) </center>